该笔记本仅用于测试中转api

In [1]:
from dotenv import load_dotenv

if not load_dotenv("../config/.env"):
    raise Exception("Failed to load the .env file")

In [2]:
from langchain_openai import ChatOpenAI, OpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = OpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

In [3]:

from langchain.chains.llm_math.base import LLMMathChain
from langchain_core.tools import Tool

llm_math_chain = LLMMathChain.from_llm(llm)
from langchain.utilities import SerpAPIWrapper
serp_api = SerpAPIWrapper()

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper = WikipediaAPIWrapper(top_k_result=1, doc_content_chars_max=100)
wikitool = WikipediaQueryRun(api_wrapper=api_wrapper)
wikitool.name = 'Wikipedia'
wikitool.description ='A wrapper around Wikipedia. Useful for when you need to answer general question about definition and the description of people, place, facrts, history etc.'


tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
    Tool(
        name="GoogleSearch",
        func=serp_api.run,
        description="This is Google search tool. Useful for searching some real time info, such as news.",
    ),
    wikitool,
]


In [4]:

from langchain.agents import create_react_agent, initialize_agent, AgentType, AgentExecutor, create_openai_tools_agent
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt_ = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to "
            "chat! But when they ask you about something that happened in real life, you may need to use tools to "
            "ensure that your answers are accurate."
            # "There are also some tools related to Gmail that allow you to view "
            # "Gmail information or send an email. Generally, you do not need to use these tools, but when users want "
            # "to process emails, you need to use them correctly."
            ,
        ),
        # MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



# Set up memory
msgs = StreamlitChatMessageHistory(key="langchain_messages")
if len(msgs.messages) == 0:
    msgs.add_ai_message("How can I help you?")

# from langchain import hub
# react_agent_ = create_react_agent(chat, tools, hub.pull("hwchase17/react"))

openai_tools_agent = create_openai_tools_agent(chat, tools, prompt_)

agent_executor = AgentExecutor(agent=openai_tools_agent, tools=tools, )

# chain_with_history = RunnableWithMessageHistory(
#     agent_executor,
#     lambda session_id: msgs,
#     input_messages_key="input",
#     output_messages_key="output",
#     history_messages_key="history",
# )

2024-05-29 12:46:45.749 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


In [5]:
agent_executor.invoke({"input": "搜索一下武汉明天的天气"},)

{'input': '搜索一下武汉明天的天气',
 'output': '根据搜索结果显示，明天武汉的天气预报为多云，气温约为摄氏 29 度（华氏 84 度），降水概率为 10%，湿度为 51%，风力约为 10 英里/小时。'}

In [36]:

config = {"configurable": {"session_id": "any", "tool_call_id": "any"}, }
response = chain_with_history.invoke({"input": "搜索一下武汉明天的天气"}, config)

Parent run 4bde62e1-7882-4acd-8b4c-a86f9e66c47a not found for run aadd9e21-028c-4a8a-89b2-0e875fad0134. Treating as a root run.


ValidationError: 1 validation error for ToolAgentAction
tool_call_id
  none is not an allowed value (type=type_error.none.not_allowed)

In [15]:
from langchain import hub

react_agent = create_react_agent(chat, tools, hub.pull("hwchase17/react"))
agent_ = initialize_agent(tools=tools, llm=chat, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
mrkl = AgentExecutor(agent=react_agent, tools=tools)

In [11]:
mrkl.invoke({"input": "搜索一下武汉明天的天气"},)

{'input': '搜索一下武汉明天的天气',
 'output': 'The weather in Wuhan tomorrow is rainy with a temperature of 70°F, 80% precipitation, 86% humidity, and 7 mph wind.'}